In [2]:
#genereal code is used to implement model
# code referenced from: https://towardsdatascience.com/simple-transformers-introducing-the-easiest-bert-roberta-xlnet-and-xlm-library-58bf8c59b2a3
import pandas as pd
import numpy as np 
import pandas as pd

In [2]:
import wandb

In [3]:
#to see the results on the wandb website use the authorization key: 9ebe3fdc95147616015f272bf80ff34c96b5389a
wandb.init(project="mehul-project")

wandb: Currently logged in as: mehulsi (use `wandb login --relogin` to force relogin)


In [4]:
#reading data
data = pd.read_csv('Suicide_Detection.csv')

In [5]:
data.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [8]:
data = data[['text','class']]

In [9]:
data.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [10]:
data= pd.DataFrame({
    'text': data['text'].replace(r'\n', ' ', regex=True),
    'label':data['class']
})
data.head()

,text,label
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [11]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['label'] = labelencoder.fit_transform(data[['label']].values)

C:\Users\mehul\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
data.head()

,text,label
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,"I’m so lostHello, my name is Adam (16) and I’v...",1


In [13]:
#selecting fraction of data for the experiments
data = data.sample(frac=.20, random_state=14)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46415 entries, 163961 to 109038
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    46415 non-null  object
 1   label   46415 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 906.5+ KB


In [15]:
from sklearn.model_selection import train_test_split
#Splitting the dataset into train test split
train_val, test = train_test_split(data,test_size=0.2, random_state=2)

In [16]:
train_val.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37132 entries, 104026 to 151204
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    37132 non-null  object
 1   label   37132 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 725.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9283 entries, 105549 to 29061
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    9283 non-null   object
 1   label   9283 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 181.3+ KB


In [17]:
#splitting the dataset into train and validation set
train, val = train_test_split(train_val,test_size=0.2, random_state=2)

In [18]:
train.info()
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29705 entries, 214876 to 204546
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29705 non-null  object
 1   label   29705 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 580.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7427 entries, 143616 to 59712
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7427 non-null   object
 1   label   7427 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 145.1+ KB


In [17]:
#loading the model
from simpletransformers.classification import ClassificationModel

In [18]:
# selecting the hyperparameters
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 10,
    'num_train_epochs': 2,
    'evaluate_during_training_steps': 10,
    'save_eval_checkpoints': False,
    'use_early_stopping': True,
    'early_stopping_patience': 3,
    'train_batch_size': 64,
    'eval_batch_size': 32,
    'learning_rate': 1e-5
    'overwrite_output_dir': True,

    'wandb_project': "mehul-project",
}

In [19]:
model = ClassificationModel('bert', 'bert-base-uncased',args = train_args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [20]:
#training the model
model.train_model(train,eval_df=val)

/user/HS401/ms02790/.local/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:585: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


/user/HS401/ms02790/.local/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1426: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


/vol/hpc-software/software/Anaconda3/2020.07/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


RuntimeError: [enforce fail at inline_container.cc:262] . unexpected pos 835841536 vs 835841424

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/vol/hpc-software/software/Anaconda3/2020.07/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/vol/hpc-software/software/Anaconda3/2020.07/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/user/HS401/ms02790/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 152, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/user/HS401/ms02790/.local/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 125, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/user/HS401/ms02790/.local/lib/python3.8/site-packages/wandb/sdk/interface/interface_shared.py", line 388, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/user/HS401/ms02790/.local/lib/python3.8/site-packages/wandb/sdk/interface/interfa